# BACKTESTING ANALISYS

In [234]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
from datetime import datetime, date, timedelta
import math
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt

import statistics as s

import seaborn as sns
sns.set()
sns.set_theme()

## LECTURA DE DATOS

In [235]:
# Lectura de datos
start = datetime(1900,1,1)
end = datetime.now()
data = pdr.get_data_yahoo('BTC-USD', start, end, interval='d')

data = data[['Adj Close']].copy()
data= data.rename(columns={'Adj Close': 'btc_price'})
data['btc_return'] = data['btc_price'].pct_change()
data['day'] = data.index.day
data['weekday'] = data.index.dayofweek
data['month'] = data.index.month
data['year'] = data.index.year
data 



,btc_price,btc_return,day,weekday,month,year
Date,,,,,,
2014-09-17,457.334015,NaN,17,2,9,2014
2014-09-18,424.440002,-0.071926,18,3,9,2014
2014-09-19,394.795990,-0.069843,19,4,9,2014
2014-09-20,408.903992,0.035735,20,5,9,2014
2014-09-21,398.821014,-0.024659,21,6,9,2014
...,...,...,...,...,...,...
2021-11-17,60368.011719,0.003437,17,2,11,2021
2021-11-18,56942.136719,-0.056750,18,3,11,2021
2021-11-19,58119.578125,0.020678,19,4,11,2021


In [236]:
data.to_csv('datos_btc.csv')

## FUNCIONES

## Calculo del CAGR

In [237]:

estrategies = ['_E1', '_E2', '_E3']

def cagr(df):
    '''
    Retorna un dataframe con el cálculo del CAGR por estrategia de compra
    '''
    cagr_results = []
    for e in estrategies: 
        df = df.copy()
        df = df.sort_index(ascending=True)
        Begining_value = df['usdt_invested'+e].sum()
        Ending_Value = df['btc_amount'+e].sum()*df['btc_price'][-1]
        total_ret = Ending_Value/Begining_value

        start = datetime.utcfromtimestamp(df.index.values[0].astype('O')/1e9)
        end = datetime.utcfromtimestamp(df.index.values[-1].astype('O')/1e9)
        period_years = (end - start).days / 365.25
        CAGR = round((total_ret)**(1/period_years)-1, 4)
        cagr_results.append(CAGR)
        #print(f'Estrategia{e}: ', round(CAGR,3))
    cagr_results = pd.DataFrame({'cagr': cagr_results}, index=['DCA_simple', 'DCA_variable', 'DCA_variable_caídas'])
    return cagr_results

## Estrategias DCA

In [238]:

def DCA_simple(df, usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según la frecuencia de compra
   '''
   for x in df.index:
      row = df.loc[x]
      df.at[x,'usdt_invested_E1'] = usdt
    
   df['btc_amount_E1'] = df['usdt_invested_E1']/df['btc_price']
   return df.head(20)

In [239]:

def DCA_variable(df, usdt, var_usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos (según la variación del retorno) y la cantidad de btc obtenidos.
   '''
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] < -(0.05)):
         df.at[x,'usdt_invested_E2'] = usdt*(1+var_usdt)
      elif(row['btc_return'] > 0.05):    
         df.at[x,'usdt_invested_E2'] = usdt*(1-var_usdt)
      else:
         df.at[x, 'usdt_invested_E2'] = usdt
   df['btc_amount_E2'] = df['usdt_invested_E2']/df['btc_price']
   return df.head(20)

In [240]:

def DCA_variable_caida(df, usdt):
   '''
   Retorna un dataframe con la cantidad de usdt invertidos según la variación del retorno y la cantidad de btc obtenidos
   '''
   for x in df.index:
      row = df.loc[x]
      if(row['btc_return'] < -0.0):
         df.at[x,'usdt_invested_E3'] = usdt*(1+row['btc_return'])
      else:
         df.at[x, 'usdt_invested_E3'] = usdt
   df['btc_amount_E3'] = df['usdt_invested_E3']/df['btc_price']
   return df.head(20)

In [241]:
# DataFrame semanal
usdt = 50
df = data.copy()
#df = df[(df['year'] == 2021) & (df['weekday']==0)] 
df = df[df['weekday']==0] 
df['btc_return'] = df['btc_price'].pct_change() 
DCA_simple(df, usdt)
DCA_variable(df, usdt,1)
DCA_variable_caida(df,usdt)


df.head()

,btc_price,btc_return,day,weekday,month,year,usdt_invested_E1,btc_amount_E1,usdt_invested_E2,btc_amount_E2,usdt_invested_E3,btc_amount_E3
Date,,,,,,,,,,,,
2014-09-22,402.152008,NaN,22,0,9,2014,50.0,0.124331,50.0,0.124331,50.000000,0.124331
2014-09-29,375.467010,-0.066355,29,0,9,2014,50.0,0.133167,100.0,0.266335,46.682225,0.124331
2014-10-06,330.079010,-0.120884,6,0,10,2014,50.0,0.151479,100.0,0.302958,43.955794,0.133167
2014-10-13,390.414001,0.182790,13,0,10,2014,50.0,0.128069,0.0,0.000000,50.000000,0.128069
2014-10-20,382.845001,-0.019387,20,0,10,2014,50.0,0.130601,50.0,0.130601,49.030644,0.128069


In [242]:
# Cálculo del CAGR por estrategia
cagr_btc = round(cagr(df),3)
cagr_btc


,cagr
DCA_simple,0.769
DCA_variable,0.790
DCA_variable_caídas,0.769


In [243]:

def calculate_return(df):
    # retorno E1: DCA Simple
    total_btc_amount_E1 = df['btc_amount_E1'].sum()
    total_usdt_invested_E1 = df['usdt_invested_E1'].sum()
    total_usdt_obtained_E1 = round(total_btc_amount_E1*df['btc_price'][-1],3)
    total_return_E1 = round((total_usdt_obtained_E1/total_usdt_invested_E1-1)*100,2)
    
    # retorno E1: DCA Variable
    total_btc_amount_E2 = df['btc_amount_E2'].sum()
    total_usdt_invested_E2 = df['usdt_invested_E2'].sum()
    total_usdt_obtained_E2 = round(total_btc_amount_E2*df['btc_price'][-1],3)
    total_return_E2 = round((total_usdt_obtained_E2/total_usdt_invested_E2-1)*100,2)


   # retorno E1: DCA Variable en caídas
    total_btc_amount_E3 = df['btc_amount_E3'].sum()
    total_usdt_invested_E3 = df['usdt_invested_E3'].sum()
    total_usdt_obtained_E3 = round(total_btc_amount_E3*df['btc_price'][-1],3)
    total_return_E3 = round((total_usdt_obtained_E3/total_usdt_invested_E3-1)*100,2)

    results = pd.DataFrame({'total_usdt_invested': [total_usdt_invested_E1, total_usdt_invested_E2, total_usdt_invested_E2],
                             'total_usdt_obtained': [total_usdt_obtained_E1, total_usdt_obtained_E2, total_usdt_obtained_E3],
                             'total_return': [total_return_E1, total_return_E2, total_return_E3]}, 
                             index=['DCA_simple', 'DCA_variable', 'DCA_variable_caídas'])
    return results
  

In [244]:
#retorno total por compra semanal
calculate_return(df)

,total_usdt_invested,total_usdt_obtained,total_return
DCA_simple,18700.0,1113937.049,5856.88
DCA_variable,16750.0,1087261.873,6391.12
DCA_variable_caídas,16750.0,1082876.189,5862.32


In [245]:
# DataFrame semanal
years = data['year'].unique().tolist()
usdt = 50

for i in years:

    df = data.copy()
    df = df[(df['year'] == i) & (df['weekday']==0)] 
    df['btc_return'] = df['btc_price'].pct_change() 
    DCA_simple(df, usdt)
    DCA_variable(df, usdt,1)
    DCA_variable_caida(df,usdt)
    retornos = calculate_return(df)
    print('_____________________________________________________________________________________')
    print(f'Año: {i}', retornos)
   

_____________________________________________________________________________________
Año: 2014                      total_usdt_invested  total_usdt_obtained  total_return
DCA_simple                         750.0              652.005        -13.07
DCA_variable                       850.0              760.018        -10.59
DCA_variable_caídas                850.0              625.930        -13.23
_____________________________________________________________________________________
Año: 2015                      total_usdt_invested  total_usdt_obtained  total_return
DCA_simple                        2600.0             4157.149         59.89
DCA_variable                      2400.0             4024.234         67.68
DCA_variable_caídas               2400.0             4034.999         59.52
_____________________________________________________________________________________
Año: 2016                      total_usdt_invested  total_usdt_obtained  total_return
DCA_simple                  